<a href="https://colab.research.google.com/github/aobd1356/Homework/blob/main/EX05_04_Reply_Template_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install -q pyngrok flask_ngrok # for colab
!pip install -q flask line-bot-sdk

In [5]:
# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
    ngrok.connect(5000)
    run_with_ngrok(app)

from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import (
    Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest,
    TextMessage,
    TemplateMessage, ConfirmTemplate, MessageAction,
    CarouselTemplate,
    CarouselColumn,
    URIAction,
    PostbackAction
)

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('access_token'))
handler = WebhookHandler(userdata.get('Channel_secret'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action=event.message.text
        if action=="confirm":
              template=ConfirmTemplate(
                  text="你喜歡Bang Dream嗎？",
                  actions=[
                      MessageAction(label="是", text="我很喜歡"),
                      MessageAction(label="否", text="其實我超級喜歡")
                  ]
              )
              reply=TemplateMessage(
              alt_text="這是確認視窗",
              template=template
          )
        elif action=='carousel':
          carousel_template=CarouselTemplate(
                columns=[
                  CarouselColumn(
                    thumbnail_image_url='https://zh.wikipedia.org/zh-tw/%E6%A8%B1%E5%B7%AB%E5%A5%B3#/media/File:%E6%AB%BB%E5%B7%AB%E5%A5%B3.png',
                    title='櫻巫女',
                    text='日本vtuber',
                    actions=[
                        URIAction(label='維基百科', uri='https://zh.wikipedia.org/zh-tw/%E6%A8%B1%E5%B7%AB%E5%A5%B3'),
                        URIAction(label='YOUTUBE', uri='https://www.youtube.com/@SakuraMiko'),
                        MessageAction(label="投票",text="我比較喜歡櫻巫女")
                    ]
                  ),
                  CarouselColumn(
                    thumbnail_image_url='https://zh.wikipedia.org/wiki/%E6%98%9F%E8%A1%97%E5%BD%97%E6%98%9F#/media/File:%E6%98%9F%E8%A1%97%E5%BD%97%E6%98%9F.png',
                    title='星街彗星',
                    text='日本vtuber',
                    actions=[
                        URIAction(label='維基百科', uri='https://zh.wikipedia.org/wiki/%E6%98%9F%E8%A1%97%E5%BD%97%E6%98%9F'),
                        URIAction(label='YOUTUBE', uri='https://www.youtube.com/@HoshimachiSuisei'),
                        MessageAction(label="投票",text="我比較喜歡星街彗星")
                    ]
                  )
                ]
          )
          reply=TemplateMessage(
              alt_text="這是輪播視窗",
              template=carousel_template
              )
        else:
          reply=Textmessage(text=event.message.text)

        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[
                    reply
                ]
            )
        )



ngrok_start() # for colab
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://06ac-34-133-215-196.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [13/May/2025 07:48:17] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 07:48:53] "POST /callback HTTP/1.1" 200 -
ERROR:__main__:Exception on /callback [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 919, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/flask/app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^